In [4]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# Load the training data
train_data = pd.read_csv('/kaggle/input/hackathon-online-cryptocurrency-prediction/train.csv')
train_data = train_data[['price']].values

# Scale price between 0 - 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaler_train = scaler.fit_transform(train_data)

# Define sequence length and split the data
seq_len = 4000  # Number of previous prices to consider
train_X, train_Y = [], []
for i in range(seq_len, len(scaler_train)):
    train_X.append(scaler_train[i - seq_len:i])
    train_Y.append(scaler_train[i])
train_X, train_Y = np.array(train_X), np.array(train_Y)

# Reshape the input
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

# Split the data into train and test sets
train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y, test_size=0.2, random_state=42)

# Define and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train_X, train_Y, epochs=45, batch_size=32)

# Calculate mean absolute error (MAE) on the test set
y_pred = model.predict(test_X)
mae = mean_absolute_error(test_Y, y_pred)
print('MAE:', mae)

# Prepare predict data
last_data = train_data[-seq_len:]
scaled_last_data = scaler.transform(last_data)
test_X = np.array([scaled_last_data[:, 0]])
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 1))

submit_data = pd.read_csv('/kaggle/input/hackathon-online-cryptocurrency-prediction/test.csv')

# Predict
predicted_prices = []
for i in range(len(submit_data)):
    predicted_price = model.predict(test_X)
    predicted_prices.append(predicted_price[0][0])
    
    next_data = np.append(test_X[0][1:], predicted_price[0][0])
    test_X = np.reshape(next_data, (1, seq_len, 1))
    
# Inverse transform predicted prices
predicted_prices = np.array(predicted_prices)
predicted_prices = np.reshape(predicted_prices, (len(predicted_prices), 1))
predicted_prices = scaler.inverse_transform(predicted_prices)

# Create submit.csv
submit_data = pd.read_csv('/kaggle/input/hackathon-online-cryptocurrency-prediction/test.csv')
submit_data['price'] = predicted_prices
submit_data.to_csv('submit1.csv', index=False)

[[   1]
 [   2]
 [   3]
 ...
 [4998]
 [4999]
 [5000]]
[[10.383]
 [ 6.163]
 [ 8.812]
 ...
 [ 0.697]
 [ 0.687]
 [ 0.756]]
